In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import uproot
import tqdm
import shap

/home/shounak/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_list=[]
for filenumber in tqdm.tqdm(range(1,10)):
# Open the .root file
    filePath=f'../36_files/JetNtuple_RunIISummer16_13TeV_MC_{filenumber}.root'
    file = uproot.open(filePath)
# Access the tree structure
    tree = file['AK4jets/jetTree']
# Define the columns you want to read
    columns = ['isPhysG', 'isPhysUDS','QG_mult','QG_ptD','QG_axis2','jetPt','jetEta','jetQGl','jetMass','jetGirth','jetArea','jetChargedHadronMult','jetNeutralHadronMult','jetChargedMult','jetNeutralMult']
    df=tree.arrays(columns, library='pd')
    df_list.append(df)
df=pd.concat(df_list,ignore_index='true')
# First remove all the other types of jets from the data except the gluon and the light quark (UDS) jets
df = df[(df.isPhysG==1) | (df.isPhysUDS==1)].reset_index()
#selecting only a range of pt 
df_pt_ranged=df[(df.jetPt>30) & (df.jetPt<600)]
df=df_pt_ranged

from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)

train_y = train.isPhysUDS
test_y = test.isPhysUDS
# training and test variables for each training variable
#input_features = ['QG_mult','QG_ptD','QG_axis2','jetPt','jetEta','jetMass','jetGirth','jetArea','jetChargedHadronMult','jetNeutralHadronMult']
input_features = ['QG_mult','QG_ptD','QG_axis2','jetMass','jetGirth','jetArea','jetChargedHadronMult','jetNeutralHadronMult']

train_x_var=[]
test_x_var=[]

for input in input_features:
    train_x = train[[input]]
    test_x = test[[input]]
 # Normalize the features
    train_x_mean = train_x.mean()
    train_x_std = train_x.std()
    exec(f'train_x_{input}= (train_x - train_x_mean) / train_x_std')
    exec(f'test_x_{input} = (test_x - train_x_mean) / train_x_std')
    exec(f'train_x_var.append(train_x_{input})')
    exec(f'test_x_var.append(test_x_{input})')
train_x=train[input_features]
test_x=test[input_features]


100%|██████████| 9/9 [00:02<00:00,  4.10it/s]


In [4]:
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout
from sklearn.utils import class_weight
from keras.callbacks import EarlyStopping


2024-10-28 11:24:18.164627: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-28 11:24:18.328975: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-28 11:24:18.329102: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-28 11:24:18.354578: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 11:24:18.422652: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-28 11:24:18.423862: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [26]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.callbacks import EarlyStopping
from sklearn.utils import class_weight

from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)

# Constants and Callback
input_features = ['QG_mult','QG_ptD','QG_axis2','jetMass','jetGirth','jetArea','jetChargedHadronMult','jetNeutralHadronMult']
epoch_number = 200
callback = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=8, verbose=1, mode='auto', restore_best_weights=False)

# Dataset
train_x_var, test_x_var, model_list = {}, {}, {}

# Model architecture function
def architecture(model_name, input_shape):
    model_name.add(Input(shape=(input_shape,)))
    model_name.add(Dropout(0.2))
    model_name.add(Dense(100, kernel_initializer='normal', activation='relu'))
    model_name.add(Dropout(0.2))
    model_name.add(Dense(50, kernel_initializer='normal', activation='relu'))
    model_name.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model_name.compile(optimizer='Nadam', loss='binary_crossentropy', metrics=['accuracy'])

# Fitting function
def fitting(model_name, train_x, train_y, class_weights, model_name_str):
    model_name.fit(
        train_x, train_y,
        epochs=epoch_number,
        batch_size=128,
        class_weight=class_weights,
        validation_split=0.2,
        shuffle=True,
        verbose=1,
        callbacks=[callback]
    )
    model_name.save(f"{model_name_str}.h5")

# Prepare Data: Normalize each feature and store in dictionaries
for input_feature in input_features:
    # Normalize train and test data
    train_x_feature = train[[input_feature]]
    test_x_feature = test[[input_feature]]
    train_x_mean = train_x_feature.mean()
    train_x_std = train_x_feature.std()

    # Normalized feature
    train_x_var[input_feature] = (train_x_feature - train_x_mean) / train_x_std
    test_x_var[input_feature] = (test_x_feature - train_x_mean) / train_x_std

    # Initialize a model for each feature
    model = Sequential()
    architecture(model, input_shape=train_x_var[input_feature].shape[1])
    model_list[input_feature] = model

# Train each model on its respective feature data
for feature_name in input_features:
    # Extract train and test data for this feature
    train_x = train_x_var[feature_name]
    test_x = test_x_var[feature_name]
    train_y = train['isPhysUDS']  # Replace with your target column name

    # Compute class weights for this feature's training labels
    classes = np.unique(train_y)
    class_weights = dict(enumerate(class_weight.compute_class_weight('balanced', classes=classes, y=train_y)))

    # Train and save the model
    fitting(model_list[feature_name], train_x, train_y, class_weights, f"model_{feature_name}")


Epoch 1/200
2682/2682 [==============================] - 7s 2ms/step - loss: 0.6596 - accuracy: 0.6069 - val_loss: 0.6484 - val_accuracy: 0.6137
Epoch 2/200
2682/2682 [==============================] - 6s 2ms/step - loss: 0.6588 - accuracy: 0.6070 - val_loss: 0.6511 - val_accuracy: 0.6015
Epoch 3/200
2682/2682 [==============================] - 6s 2ms/step - loss: 0.6588 - accuracy: 0.6037 - val_loss: 0.6558 - val_accuracy: 0.5691
Epoch 4/200
2682/2682 [==============================] - 5s 2ms/step - loss: 0.6586 - accuracy: 0.6053 - val_loss: 0.6346 - val_accuracy: 0.6137
Epoch 5/200
2682/2682 [==============================] - 6s 2ms/step - loss: 0.6590 - accuracy: 0.6060 - val_loss: 0.6510 - val_accuracy: 0.6015
Epoch 6/200
2682/2682 [==============================] - 6s 2ms/step - loss: 0.6583 - accuracy: 0.6012 - val_loss: 0.6470 - val_accuracy: 0.6137
Epoch 7/200
2682/2682 [==============================] - 6s 2ms/step - loss: 0.6589 - accuracy: 0.6013 - val_loss: 0.6403 - val_ac

/home/shounak/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/200
2682/2682 [==============================] - 9s 3ms/step - loss: 0.6091 - accuracy: 0.6666 - val_loss: 0.6026 - val_accuracy: 0.6885
Epoch 2/200
2682/2682 [==============================] - 8s 3ms/step - loss: 0.6076 - accuracy: 0.6638 - val_loss: 0.5877 - val_accuracy: 0.7103
Epoch 3/200
2682/2682 [==============================] - 7s 3ms/step - loss: 0.6076 - accuracy: 0.6669 - val_loss: 0.5848 - val_accuracy: 0.7159
Epoch 4/200
2682/2682 [==============================] - 7s 3ms/step - loss: 0.6074 - accuracy: 0.6611 - val_loss: 0.5793 - val_accuracy: 0.7061
Epoch 5/200
2682/2682 [==============================] - 8s 3ms/step - loss: 0.6076 - accuracy: 0.6652 - val_loss: 0.5955 - val_accuracy: 0.6950
Epoch 6/200
2682/2682 [==============================] - 7s 3ms/step - loss: 0.6071 - accuracy: 0.6662 - val_loss: 0.5947 - val_accuracy: 0.7021
Epoch 7/200
2682/2682 [==============================] - 7s 2ms/step - loss: 0.6080 - accuracy: 0.6663 - val_loss: 0.5797 - val_ac

/home/shounak/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/200
2682/2682 [==============================] - 11s 3ms/step - loss: 0.6051 - accuracy: 0.6850 - val_loss: 0.5692 - val_accuracy: 0.7359
Epoch 2/200
2682/2682 [==============================] - 9s 3ms/step - loss: 0.6009 - accuracy: 0.7041 - val_loss: 0.5728 - val_accuracy: 0.7242
Epoch 3/200
2682/2682 [==============================] - 9s 3ms/step - loss: 0.6003 - accuracy: 0.6967 - val_loss: 0.5614 - val_accuracy: 0.7327
Epoch 4/200
2682/2682 [==============================] - 9s 3ms/step - loss: 0.5996 - accuracy: 0.6942 - val_loss: 0.5681 - val_accuracy: 0.7233
Epoch 5/200
2682/2682 [==============================] - 9s 3ms/step - loss: 0.5989 - accuracy: 0.6907 - val_loss: 0.5714 - val_accuracy: 0.7205
Epoch 6/200
2682/2682 [==============================] - 9s 3ms/step - loss: 0.5996 - accuracy: 0.6874 - val_loss: 0.5750 - val_accuracy: 0.7115
Epoch 7/200
2682/2682 [==============================] - 9s 3ms/step - loss: 0.5988 - accuracy: 0.6929 - val_loss: 0.5689 - val_a

...

/home/shounak/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2682/2682 [==============================] - 4s 1ms/step - loss: 0.6654 - accuracy: 0.5972 - val_loss: 0.6509 - val_accuracy: 0.6164
Epoch 2/200
2682/2682 [==============================] - 3s 1ms/step - loss: 0.6649 - accuracy: 0.5931 - val_loss: 0.6486 - val_accuracy: 0.6164
Epoch 3/200
2682/2682 [==============================] - 9s 3ms/step - loss: 0.6648 - accuracy: 0.5973 - val_loss: 0.6531 - val_accuracy: 0.6164
Epoch 4/200
2682/2682 [==============================] - 9s 3ms/step - loss: 0.6643 - accuracy: 0.6067 - val_loss: 0.6582 - val_accuracy: 0.5809
Epoch 5/200
2682/2682 [==============================] - 8s 3ms/step - loss: 0.6646 - accuracy: 0.5956 - val_loss: 0.6512 - val_accuracy: 0.5992
Epoch 6/200
2682/2682 [==============================] - 8s 3ms/step - loss: 0.6648 - accuracy: 0.5983 - val_loss: 0.6515 - val_accuracy: 0.5992
Epoch 7/200
2682/2682 [==============================] - 8s 3ms/step - loss: 0.6648 - accuracy: 0.5959 - val_loss: 0.6586 - val_accuracy: 0.58

/home/shounak/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/200
2682/2682 [==============================] - 10s 3ms/step - loss: 0.6925 - accuracy: 0.5605 - val_loss: 0.6930 - val_accuracy: 0.5208
Epoch 2/200
2682/2682 [==============================] - 8s 3ms/step - loss: 0.6925 - accuracy: 0.5652 - val_loss: 0.6890 - val_accuracy: 0.6076
Epoch 3/200
2682/2682 [==============================] - 9s 3ms/step - loss: 0.6925 - accuracy: 0.5716 - val_loss: 0.6908 - val_accuracy: 0.6076
Epoch 4/200
2682/2682 [==============================] - 8s 3ms/step - loss: 0.6925 - accuracy: 0.5666 - val_loss: 0.6898 - val_accuracy: 0.6076
Epoch 5/200
2682/2682 [==============================] - 9s 3ms/step - loss: 0.6925 - accuracy: 0.5662 - val_loss: 0.6890 - val_accuracy: 0.6076
Epoch 6/200
2682/2682 [==============================] - 9s 3ms/step - loss: 0.6925 - accuracy: 0.5704 - val_loss: 0.6892 - val_accuracy: 0.6200
Epoch 7/200
2682/2682 [==============================] - 8s 3ms/step - loss: 0.6925 - accuracy: 0.5601 - val_loss: 0.6938 - val_a

/home/shounak/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
